# Tennis Match Data Analysis 

For an introduction to the tennis match data set, visit 
https://docs.google.com/document/d/1pUpK76tkvP09V1ptHHEv77gEnzyZLIMGq2vNobyICGE/edit

## How to load and process tennis data?

Please note the following:
- It takes a long time to process the point data. So we avoid processing the point data if possible. 
- The raw match data is missing many match_ids. Some match_ids in the point data are not in the original match data. So we need to synchronize them before using the match data. 
- Both match data and point data need cleaning. 
- We need to process the point data to generate new data columns. 

Here are the steps for loading data: 
- Fastest: Load the processed point data and processed math data, and then you can start your data analysis. 
- Slow: If you download new data from GitHub, you need to clean and process both the raw match data and raw point data. 
- Faster: If you have changed some of the match data cleaning and processing functions, you need to load the raw match data and process it. Load the processed point data, and synchronized them. The match data will be updated. The point data is unchanged. Save the processed match data.

## 1. Commonly used functions for match data

Download the raw match data from https://github.com/JeffSackmann/tennis_MatchChartingProject.
<br>For male players, download charting-m-matches.csv. It's better to convert it to a .xls file. 

Some commonly used functions are specified in this section.

## Functions for match data cleaning

In [5]:
import pandas as pd

In [6]:
def clean_match_data(df_matches): 
    
    df_matches = df_matches.copy()
    
    # There are several duplicated matches. Remove them. 
    df_matches.drop_duplicates(subset="match_id", inplace=True)

    # Data cleaning
    df_matches["Player 1"] = df_matches["Player 1"].str.strip()
    df_matches["Player 2"] = df_matches["Player 2"].str.strip()
    df_matches["Player 1"] = df_matches["Player 1"].replace(to_replace=r"\s+", value="_", regex=True)
    df_matches["Player 2"] = df_matches["Player 2"].replace(to_replace=r"\s+", value="_", regex=True)
    df_matches["Player 1"] = df_matches["Player 1"].str.lower()
    df_matches["Player 2"] = df_matches["Player 2"].str.lower()
    df_matches["Pl 1 hand"] = df_matches["Pl 1 hand"].str.strip()
    df_matches["Pl 2 hand"] = df_matches["Pl 2 hand"].str.strip()
    df_matches["Pl 1 hand"] = df_matches["Pl 1 hand"].str.lower()
    df_matches["Pl 2 hand"] = df_matches["Pl 2 hand"].str.lower()
    df_matches["Tournament"] = df_matches["Tournament"].str.strip()
    df_matches["Tournament"] = df_matches["Tournament"].str.lower()
    df_matches["Tournament"] = df_matches["Tournament"].replace(to_replace=r"\s+", value="_", regex=True)
    df_matches["Surface"] = df_matches["Surface"].str.strip()
    df_matches["Surface"] = df_matches["Surface"].str.lower()
    df_matches["Gender"] = df_matches["Surface"].str.strip()
    df_matches["Gender"] = df_matches["Surface"].str.lower()


    df_matches = df_matches.astype({"Date": "str"})
    # df_matches = df_matches.astype({"Best of": "int32"})
    df_matches["Year"] = df_matches["Date"].str.extract(r"^(\d{4})")

    df_matches["Surface"] = df_matches["Surface"].fillna("unknown")
    df_matches["Tournament"] = df_matches["Tournament"].fillna("unknown")
    df_matches["Best of"] = df_matches["Best of"].fillna(0)
    df_matches["Player 1"] = df_matches["Player 1"].fillna("unknown")
    df_matches["Player 2"] = df_matches["Player 2"].fillna("unknown")
    df_matches["Pl 1 hand"] = df_matches["Pl 1 hand"].fillna("unknown")
    df_matches["Pl 2 hand"] = df_matches["Pl 2 hand"].fillna("unknown")
    df_matches["Round"] = df_matches["Round"].fillna("unknown")

    return df_matches

#     df_matches.info()
#     print()

#     print("There are %d matches in the database." % len(df_matches))
#     # df_matches.head(30)

#     df_matches.to_excel("charting-m-matches-cleaned.xls")

In [7]:
# How many matches are there for each player in the database?
def get_player_match_count(df_matches):
    players = df_matches["Player 1"].append(df_matches["Player 2"])
    player_match_count = players.value_counts()
    
    return player_match_count 

In [8]:
def get_players(df_matches):
    players = df_matches["Player 1"].append(df_matches["Player 2"]).unique()
    
    return players

In [9]:
# clean the handedness columns because there are errors. Create a table of players and their handednesses.
def clean_handedness_match_data(df_matches):
    
    df_matches = df_matches.copy()
    
    df_players_handedness = pd.DataFrame()
    
    players = get_players(df_matches)

    # Check every player and check if there is an error in this player's handedness.
    # Some players may be marked as righthanded in some matches and lefthanded in other matches.
    # A playe cannot have two handedness. 
    for player in players:

        #these two lines select the columns "Pl 1 hand" and "Pl 2 hand" from df_matches for each player
        selected_player1_hand = df_matches[(df_matches["Player 1"] == player)]["Pl 1 hand"]
        selected_player2_hand = df_matches[(df_matches["Player 2"] == player)]["Pl 2 hand"]

        #correct_handedness needs to have an initial value
        #so that if there is no error in handedness,
        #we can just use the default handedness
        #if there is an error, it will display later in the if function
        if len(selected_player1_hand) > 0:
            # Use the majority of the handedness as a default handedness. 
            correct_handedness = selected_player1_hand.value_counts().idxmax()
        elif len(selected_player2_hand) > 0:
            correct_handedness = selected_player2_hand.value_counts().idxmax()
        else:
            correct_handedness = None

        #assign variable so that yoou do not have to continuously use the value_counts() function
        selected_player_hand_value_counts = selected_player1_hand.value_counts()

        #if the player has more than one handedness in Pl 1 hand column,
        #replace the wrong hand with correct hand
        if (len(selected_player_hand_value_counts) > 1):
            print(player)
            print(selected_player_hand_value_counts)

            #finding which hand occurs more frequently for the player
            #this is also the correct handedness for that player
            correct_handedness = selected_player_hand_value_counts.idxmax()

            #finding which hand occurs less frequently for the player
            #this is also the wrong handedness for that player
            wrong_handedness = selected_player_hand_value_counts.idxmin()

            #must save back to the original column and replace wrong_hand with correct_hand
            df_matches.loc[(df_matches["Player 1"] == player), "Pl 1 hand"] = \
                df_matches.loc[(df_matches["Player 1"] == player), "Pl 1 hand"].replace(
                wrong_handedness, correct_handedness)

        #Do the same for the "Pl 2 hand" column
        selected_player_hand_value_counts = selected_player2_hand.value_counts()
        if (len(selected_player_hand_value_counts) > 1):
            print(player)
            print(selected_player_hand_value_counts)
            correct_handedness = selected_player_hand_value_counts.idxmax()
            wrong_handedness = selected_player_hand_value_counts.idxmin()
            df_matches.loc[(df_matches["Player 2"] == player), "Pl 2 hand"] = \
                df_matches.loc[(df_matches["Player 2"] == player), "Pl 2 hand"].replace(
                wrong_handedness, correct_handedness)

        row = {"player": player, "handedness": correct_handedness}
        df_players_handedness = df_players_handedness.append(row, ignore_index=True)

    # df_players_handedness now has the correct handedness for each player. 
#     print(df_players_handedness)
    
    return df_matches, df_players_handedness

In [10]:
# Find out how many right handed and left handed players are in the database
def get_handedness_count(df_player_handedness): 
    unique_handed_players = df_players_handedness["handedness"]
    handedness_count = unique_handed_players.value_counts()
    return handedness_count

In [11]:
# Find the number of matches per tournament
def get_match_count_by_tournament(df_matches):
    tournaments = df_matches["Tournament"]
    tournaments_value_counts = tournaments.value_counts(dropna=False)
    return tournaments_value_counts

In [12]:
# Input: match data
# Output: A dataframe with each player's match count for each tournament
def get_player_match_count_by_tournament(data):
    
    data = data.copy()
    
    # Selecting just the columns, not the rows
    player1_tournament = data[["Player 1", "Tournament", "Year"]]
    
    player2_tournament = data[["Player 2", "Tournament", "Year"]]
    
    # Renaming the columns to make appending easier
    # This way, the players will be added to one column
    player1_tournament.rename(columns={"Player 1": "Player"}, inplace=True)
    player2_tournament.rename(columns={"Player 2": "Player"}, inplace=True)
    
    player_tournament = player1_tournament.append(player2_tournament, ignore_index=True)
#     print(player_tournament_sorting)
    # Grouping the players by Player, Tournament, and Year, combines similar matches with same value in column
    player_tournament_count = player_tournament.groupby(["Player", "Tournament", "Year"])["Player"].count()
    # Must-do, make "Player" and "Tournament" and "Year" column names
    
    # Count is now the index column with original Series values
    player_tournament_count = player_tournament_count.reset_index(name="Count")
    
    # Sort values from greatest to least
    player_tournament_count.sort_values(by=["Count"], ascending=False, inplace=True)
    return player_tournament_count

In [13]:
# This function counts how many times two players have played within the database,
# regardless of who served first or whether they appeared just in "Player 1" or "Player 2"
def get_player_head_to_head_count(data):
    data = data.copy()
    
    # checking how many matches "A" as player 1 has played against "B" as player 2
    player_hth_count = data.groupby(["Player 1", "Player 2"])["Player 1"].count()
    player_hth_count = player_hth_count.reset_index(name="Count")
    player_hth_count.sort_values(by=["Count"], ascending=False, inplace=True)
#     print(player_hth_count.head(20))
    # Making a copy of the data so the original is not affected
    # The data in the copy has already gone through groupby(), so no need to do groupby() again
    player_hth_count2 = player_hth_count.copy()
    # Swapping column names to count how many matches "B" as player 1 has played against "A" as player 2
    player_hth_count2.rename(columns={"Player 1": "Player 2", "Player 2": "Player 1"}, inplace=True)
    player_hth_count = player_hth_count.append(player_hth_count2)
    # Sum up in "Count" because that is the number of times they have played
    player_hth_count = player_hth_count.groupby(["Player 1", "Player 2"])["Count"].sum()
    # Reset the index so "Player 1" and "Player 2" become real columns, not indices
    # "Count" now becomes the new index column
    player_hth_count = player_hth_count.reset_index(name="Count")
    player_hth_count.sort_values(by=["Count"], ascending=False, inplace=True)
    
    return player_hth_count

In [14]:
# For each tournament, count the number of editions of this tournament appears in the database. 
# For example, how many unique "US Open" tournaments appear in the database.
def get_tournament_count(data):
    data = data.copy()
    
    tournament_year_count = data.groupby(["Tournament", "Year"])["Tournament"].count()
    # Must-do, make "Player" and "Tournament" and "Year" column names
    # Count is now the index column with original Series values
    tournament_year_count = tournament_year_count.reset_index(name="Count")
    # Sort values from greatest to least
    tournament_year_count.sort_values(by=["Count"], ascending=False, inplace=True)
    
    return tournament_year_count

In [15]:
# Select matches based on different conditions
# This is one of the most important functions
def get_matches(data, match_id = None, player1=None, player2=None, tournament=None, surface=None, 
                   player1_handedness=None, player2_handedness=None, best_of=None, date=None):
    df_results = data.copy()
    
    if match_id != None:
        df_results = df_results.loc[df_results["match_id"] == match_id]
    
    if player1 != None:
        df_results = df_results.loc[(df_results["Player 1"].str.contains(player1)) | (df_results["Player 2"].str.contains(player1))]
    
    if player2 != None:
        df_results = df_results.loc[(df_results["Player 1"].str.contains(player2)) | (df_results["Player 2"].str.contains(player2))]
        
    if tournament != None:
        df_results = df_results.loc[(df_results["Tournament"].str.contains(tournament))]
        
    if surface != None:
        df_results = df_results.loc[(df_results["Surface"].str.contains(surface))]
    
    if (player1 != None) & (player2_handedness != None):
        df_results = df_results.loc[((df_results["Player 1"].str.contains(player1)) & (df_results["Pl 2 hand"] == player2_handedness)) | 
                                   ((df_results["Player 2"].str.contains(player1)) & (df_results["Pl 1 hand"] == player2_handedness))]
    
    if (player2 != None) & (player1_handedness != None):
        df_results = df_results.loc[((df_results["Player 1"].str.contains(player2)) & (df_results["Pl 2 hand"] == player1_handedness)) | 
                                   ((df_results["Player 2"].str.contains(player2)) & (df_results["Pl 1 hand"] == player1_handedness))]
        
    if (best_of != None):
        df_results = df_results.loc[(df_results["Best of"] == best_of)]
        
    if (date != None):
        df_results = df_results.loc[(df_results["Date"].str.contains("^" + date))]
                                    
    return df_results
    

## 2. Commonly used functions for point-by-point data

Download the raw match data from https://github.com/JeffSackmann/tennis_MatchChartingProject.
<br>For male players, download charting-m-points.csv. 

<br> When you download very large csv files and the files can't be read, you can use excel to save it as "CSV UTF-8(Comma delimited)" format, then you can read the entire spreadsheet.

Some commonly used functions are specified in this section.

## Data cleaning and pre-processing

In [85]:
def clean_point_data(df_points):
    df_points = df_points.copy()
    
    # Do we need to fill na with 0?
    df_points = df_points.astype({"rallyLen": "float64"})
    df_points = df_points.astype({"TB?": "int32"})
    df_points = df_points.astype({"Pt": "int32"})
    df_points = df_points.astype({"Set1": "int32"})
    df_points = df_points.astype({"Set2": "int32"})
    df_points = df_points.astype({"Gm1": "int32"})
    df_points = df_points.astype({"Gm2": "int32"})
    # df_points = df_points.astype({"TbSet": "int32"})
    df_points = df_points.astype({"Svr": "int32"})
    df_points = df_points.astype({"Ret": "int32"})
    
    # Excel automatically changes "1-2" to "1-Feb", etc. So we have to change them back.
    df_points["Pts"] = df_points["Pts"].str.replace("Jan", "1", case=False)
    df_points["Pts"] = df_points["Pts"].str.replace("Feb", "2", case=False)
    df_points["Pts"] = df_points["Pts"].str.replace("Mar", "3", case=False)
    df_points["Pts"] = df_points["Pts"].str.replace("Apr", "4", case=False)
    df_points["Pts"] = df_points["Pts"].str.replace("May", "5", case=False)
    df_points["Pts"] = df_points["Pts"].str.replace("Jun", "6", case=False)
    df_points["Pts"] = df_points["Pts"].str.replace("Jul", "7", case=False)
    df_points["Pts"] = df_points["Pts"].str.replace("Aug", "8", case=False)
    df_points["Pts"] = df_points["Pts"].str.replace("Sep", "9", case=False)
    df_points["Pts"] = df_points["Pts"].str.replace("Oct", "10", case=False)
    df_points["Pts"] = df_points["Pts"].str.replace("Nov", "11", case=False)
    df_points["Pts"] = df_points["Pts"].str.replace("Dec", "12", case=False)
    df_points["Pts"] = df_points["Pts"].str.replace("00", "0", case=False)

    df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Jan", "1", case=False)
    df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Feb", "2", case=False)
    df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Mar", "3", case=False)
    df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Apr", "4", case=False)
    df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("May", "5", case=False)
    df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Jun", "6", case=False)
    df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Jul", "7", case=False)
    df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Aug", "8", case=False)
    df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Sep", "9", case=False)
    df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Oct", "10", case=False)
    df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Nov", "11", case=False)
    df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Dec", "12", case=False)
    df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("00", "0", case=False)
    
    return df_points

In [ ]:
# to-do

# Split match_id and add new fields to the point data


In [58]:
# Only call this if you are processing match data downloaded from GitHub. 
# If you are loading the processed match data, you don't need to call this.

# The original match data and point data are not consistent. 
# Some matches in the original point data are not in the original matches data. 
# Need to merge them
def sync_match_and_point_data(point_data, match_data):
    point_data = point_data.copy()
    match_data = match_data.copy()
    
    print("Checking if points data and match data are consistent ... ")
    num_matches_point_data = point_data["match_id"].nunique()
    num_matches_match_data = match_data["match_id"].nunique()
    print("There are " + str(num_matches_match_data) + " match ids in the match data set.")
    print("There are " + str(num_matches_point_data) + " match ids in the point data set.")
    
    if  num_matches_point_data != num_matches_match_data:
        print("The number of match ids do not match. Trying to fix the problem ...")
        unique_match_id_match_data = set(match_data["match_id"].unique())
        unique_match_id_point_data = set(point_data["match_id"].unique())
        
#         print("The following match ids are in the point data set but not the match data set")
#         print(*(unique_match_id_point_data - unique_match_id_match_data), sep="\n")
        
        print("The following match ids are in the match data set but not the point data set. Fix it.")
        print(*(unique_match_id_match_data - unique_match_id_point_data), sep ="\n")
        
        # Find the match ids that exist in the point data but not in the match data
        new_match_id = pd.Series(list(unique_match_id_point_data - unique_match_id_match_data))
        
        # Construct a dataframe based on the match ids in the point data but not in the match data 
        df_new_match_data = pd.DataFrame()
        df_new_match_data["match_id"] = new_match_id
        # Break each match_id into individual components
        df_new_match_data[['Date','Gender', "Tournament", "Round", "Player 1", "Player 2"]] = \
            new_match_id.str.lower().str.split("-", expand = True)
        
        df_new_match_data["Year"] = df_new_match_data["Date"].str.extract(r"^(\d{4})")
        
        df_new_match_data[["Surface", "Best of", "Final TB?"]] = None
        
        # Get a list of tournaments from the original match data
        list_current_matches = match_data["Tournament"].tolist()
        
        # We will check if the tournament in each new match_id already exists in the original 
        # match data. If so, we can copy the "surface", "best of", and "final tb?" into the new 
        # match data.
        for index, row in df_new_match_data.iterrows():
            # If the current tournament is already in the original match data set, copy the surface
            # Best of, and Final TB to the new match data.
            if row["Tournament"] in list_current_matches: 
                # The current tournament may appear multiple times in the original match data
                # So you only select the first one. Use iloc[], do not use index chain. 
                df_new_match_data.loc[index, "Surface"] = \
                    match_data.loc[match_data["Tournament"] == row["Tournament"], 
                                   "Surface"].iloc[0]
                df_new_match_data.loc[index, "Best of"] = \
                    match_data.loc[match_data["Tournament"] == row["Tournament"], 
                                   "Best of"].iloc[0]
                df_new_match_data.loc[index, "Final TB?"] = \
                    match_data.loc[match_data["Tournament"] == row["Tournament"], 
                                   "Final TB?"].iloc[0]
       
        # To-do: fill player handedness 
        
        # Merge the new match data with the original match data
        df_merged_match_data = match_data.append(df_new_match_data, ignore_index=True)
        
        # Check if the merged match data is consistent with the point data. 
        num_matches_match_data = df_merged_match_data["match_id"].nunique()
        if num_matches_match_data == num_matches_point_data:
            print("Match data and point data are the same at " + str(num_matches_match_data))
            print("Problem solved")
        else:
            print("Number of matches in the match data: " + str(num_matches_match_data))
            print("Number of matches in the point data: " + str(num_matches_point_data))
            print("Still has problem")
            
        return df_merged_match_data
    else:
        print("No problem found")
        return match_data


In [18]:
import re

# Identify serve side for each point and save the results in a new column "serve_side"
def identify_serve_side(data):
    data = data.copy()
    # List all the possible scores (before serve) and their corresponding serve sides. 
    dict_serve_side = {"0-0": "deuce", 
                       "0-15": "ad", 
                      "15-0": "ad",
                      "15-15": "deuce",
                      "30-0": "deuce",
                      "0-30": "deuce",
                      "30-15": "ad",
                      "15-30": "ad",
                      "40-0": "ad",
                      "0-40": "ad",
                      "40-15": "deuce",
                      "15-40": "deuce",
                       "30-30": "deuce",
                       "40-30": "ad",
                       "30-40": "ad",
                       "40-40": "deuce",
                       "40-AD": "ad",
                       "AD-40": "ad"
                      }

    data["serve_side"] = None

    for index, row in data.iterrows():
        if data.loc[index, "TB?"] == "0":
            # Identify serve sides based on the score (before serve)
            data.loc[index, "serve_side"] = dict_serve_side[data.loc[index, "Pts"]]
        elif data.loc[index, "TB?"] == "1":
            # For tiebreak points, if the sum of the two scores (before serve) are even, it's on the deuce side.
            # If the sum of the two scores (before serve) are odd, it's on the ad side. 
    #         print(data.loc[index, "Pts"])

            # Retrieve the first score
            tb_point1_str = re.search("^(\d+)-", data.loc[index, "Pts"])
            if tb_point1_str:
                tb_point1 = int(tb_point1_str.group(1))
            # Retrieve the second score
            tb_point2_str = re.search("-(\d+)", data.loc[index, "Pts"])
            if tb_point2_str:
                tb_point2 = int(tb_point2_str.group(1))

            if ((tb_point1 + tb_point2) % 2) == 0:
                data.loc[index, "serve_side"] = "deuce"
            elif ((tb_point1 + tb_point2) % 2) == 1:
                data.loc[index, "serve_side"] = "ad"

    # data[["Pts", "TB?", "serve_side"]]
    return data

In [19]:
# Separate serve direction from serve outcome

# Identify serve direction and outcome for each serve and save the results in new columns
# "Sv1_diretion," "Sv1_outcome," "Sv2_direction," and "Sv2_outcome"
def identify_serve_direction_outcome(data):
    data = data.copy()
    
    # First serve
    # The first digit of "Sv1" is the serve direction
    data["Sv1_direction"] = data["Sv1"].str.extract(r"^(\d)")
    # If the serve direction is a character not 4, 5, or 6, set it to 0.
    data.loc[data["Sv1_direction"].isin(["4", "5", "6"]) == False, "Sv1_direction"] = "0"
    # If the serve direction is empty, fill it with 0. 
    data["Sv1_direction"].fillna(value="0", inplace=True)
    # Replace numeric code with a word. May need to keep the numbers for stats analysis. 
    data["Sv1_direction"].replace({"4": "wide", "5": "body", "6": "t", "0": "unknown"},
                                  inplace=True)

    # Retrieve serve outcome
    #whatever is inside the parentheses is what is being captured/retrieved
    data["Sv1_outcome"] = data["Sv1"].str.extract(r"^\d(.+)")
    
    # Convert code to meaningful words.
    # replace() only works if the entire string in a cell matches the pattern. 
    # It cannot do substring replacement
    data["Sv1_outcome"].replace({"n": "net", "d": "deep", "*": "ace", "w": "wide",
                                 "#": "unreturnable", "x": "wide_and_deep", 
                                 "+n": "S&V_net", "+d":"S&V_deep", "+*": "S&V_ace", 
                                "+w": "S&V_wide", "+#": "S&V_unreturnable",
                                "+x": "S&V_wide_and_deep"},
                                inplace=True)
    # If the serve outcome is empty, it means the serve is in. 
    data["Sv1_outcome"].fillna(value="in", inplace=True)

    # Second serve
    data["Sv2_direction"] = data["Sv2"].str.extract(r"^(\d)")
    data.loc[data["Sv2_direction"].isin(["4", "5", "6"]) == False, "Sv2_direction"] = "0"
 
    data["Sv2_direction"].replace({"4": "wide", "5": "body", "6": "t", "0": "unknown"}, inplace=True)

    data["Sv2_outcome"] = data["Sv2"].str.extract(r"^\d(.+)")
    # replace() only works if the entire string in a cell matches the pattern. 
    # It cannot do substring replacement
    data["Sv2_outcome"].replace({"n": "net", "d": "deep", "*": "ace", "w": "wide",
                                 "#": "unreturnable", "x": "wide_and_deep", 
                                 "+n": "S&V_net", "+d":"S&V_deep", "+*": "S&V_ace", 
                                "+w": "S&V_wide", "+#": "S&V_unreturnable",
                                "+x": "S&V_wide_and_deep"},
                                inplace=True)

    # data[["Pts", "serve_side", "Sv1", "Sv2", "Sv1_direction", "Sv2_direction", "Sv1_outcome", "Sv2_outcome"]]

    print(data["Sv1_outcome"].value_counts())
    # data.info()
    return data

In [20]:
# Identify server and returner name for each point and save the results in
# the new columns "server_name" and "returner_name."
def identify_server_returner_point_winner(data):
    data = data.copy()
    
    data["server_name"] = None
    data["returner_name"] = None
    data["point_winner_name"] = None
    data["player1"] = None
    data["player2"] = None
    data["date"] = None
    data["gender"] = None
    data["tournament"] = None
    data["round"] = None

    for index, row in data.iterrows():
        # Retrieve player and tournament information from match_id, instead of
        # df_matches because it's simpler and perhaps more accurate. 
        
        match_info = pd.Series(row["match_id"].split("-"), index = 
                               ['date','gender', "tournament", "round", "player1", "player2"])
        match_info = match_info.str.lower()
        
        if row["Svr"] == 1:
            # Save data directly to the data frame.
            # Do not save data to row["Svr"] because it will not be saved to the data frame. 
            data.at[index, "server_name"] = match_info["player1"]
            data.at[index, "returner_name"] = match_info["player2"]
        elif row["Svr"] == 2:
            data.at[index, "server_name"] = match_info["player2"]
            data.at[index, "returner_name"] = match_info["player1"]

        if row["PtWinner"] == 1:
            data.at[index, "point_winner_name"] = match_info["player1"]
        else:
            data.at[index, "point_winner_name"] = match_info["player2"]

        data.at[index, "player1"] = match_info["player1"]
        data.at[index, "player2"] = match_info["player2"]
        data.at[index, "tournament"] = match_info["tournament"]
        data.at[index, "gender"] = match_info["gender"]
        data.at[index, "date"] = match_info["date"]
        data.at[index, "round"] = match_info["round"]

    # data[["server_name", "Svr", "Serving", "Pts", "serve_side", "Sv1", "Sv2", "Sv1_direction", "Sv2_direction", "Sv1_outcome", "Sv2_outcome"]]

    # data.to_csv("fed_nadal_points.csv")
    return data

In [21]:
import re    

# Parse the Rally string and separate server shots and returner shots
# Add two new columns to the returned point data: "server_shots" and "returner_shots"
def separate_server_returner_shots(point_data):
    point_data = point_data.copy()
    
    point_data["server_shots_code"] = ""
    point_data["returner_shots_code"] = ""
    point_data["how_rally_end"] = ""

    for index, row in point_data.iterrows():
        # Only process if the Rally is not empty. 
        if pd.isnull(row["Rally"]) == False:
            
            # Retrieve the rally end code
            point_data.at[index, "how_rally_end"] = re.findall(r"[*nwdx!e@#]+$", 
                                                               row["Rally"]).pop(0)
            
            # Use a regular expression to separate each shot from the Rally string.
            # Return a list of shots
            shots = re.findall(r"[fbrsvzopuylmhijktq]+[\+\-\=;^]*\d*[*nwdx!e@#]*", row["Rally"])
#             print(shots)
            while len(shots) > 0:
                # Continuously retrieve the first shot until all the shots are retrieved
                # Remember the first shot in the Rally sequence is the service return.
                point_data.at[index, "returner_shots_code"] = (point_data.at[index, 
                                                                             "returner_shots_code"]  
                                                               + shots.pop(0) + ",")
                
                # Retrieve the server's next shot, if any. 
                if len(shots) > 0:
                    point_data.at[index, "server_shots_code"] = (point_data.at[index, 
                                                                             "server_shots_code"] 
                                                                 + shots.pop(0) + ",")
   
    # to-do: Find an easy way to convert rally code to meaningful words.
    
    return point_data
    

In [88]:
# Identify critial moments: game point, break point, set point, match point, etc.
# This function handles special cases of 5th set in Wimbledon, Roland Garros, and Austrian Open. 
def identify_critical_points(point_data, match_data):
    point_data = point_data.copy()
    
    point_data["server_critical_point"] = None
    point_data["returner_critical_point"] = None
    
    game_point_scores = ["40-0", "40-15", "40-30", "AD-40"]
    break_point_scores = ["0-40", "15-40", "30-40", "40-AD"]
    server_setup_point_scores = ["30-0", "30-15", "30-30"]
    returner_setup_point_scores = ["0-30", "15-30", "30-30"]
    deuce_point_score = ["40-40"]
    
    # To-do: This function can be simplied by saving the threshold points in variables
    # based on different tournaments and "best of 5" or "best of 3".
    # The code below will be simpler, without using so
    # many ifs, particularly for tiebreak points. 
    
    for index, row in point_data.iterrows():
        
        # "Best of" may be a float. Need to typecast to int. 
        best_of = match_data.loc[match_data["match_id"] == row["match_id"], "Best of"]
        best_of = int(best_of.iloc[0])
        
        tournament = match_data.loc[match_data["match_id"] == row["match_id"], "Tournament"]
        tournament = tournament.iloc[0]
        
        # We don't need to consider year for now.
#         date = match_data.loc[match_data["match_id"] == row["match_id"], "Date"]
#         year = int(str(date.iloc[0])[:4])
        
        if row["TB?"] == 0:
            # Not in tiebreak
            if row["Pts"] in game_point_scores:
                # Server has game point
                point_data.at[index, "server_critical_point"] = "game_point"
                
                if row["Svr"] == 1:                    
                    # In Wimbledon and Roland Garros, a player needs to win two games in a row
                    # after 5-5 to win the 5th set.                    
                    if (((row["Gm1"] == 5) & (row["Gm2"] <= 4)) | 
                        ((row["Gm1"] > 5) & (row["Gm1"] > row["Gm2"]))):
                        # Server is player1 and has a set point
                        point_data.at[index, "server_critical_point"] += ",set_point"
                        
                        if (((best_of == 3) & (row["Set1"] == 1)) | 
                            ((best_of == 5) & (row["Set1"] == 2))):
                            # Player1 has already won 1 or 2 set and now has a set point
                            point_data.at[index, "server_critical_point"] += ",match_point"
                else:
                    if (((row["Gm2"] == 5) & (row["Gm1"] <= 4)) | 
                        ((row["Gm2"] > 5) & (row["Gm2"] > row["Gm1"]))):
                        # Server is player2 and has a set point
                        point_data.at[index, "server_critical_point"] += ",set_point"
                        
                        if (((best_of == 3) & (row["Set2"] == 1)) | 
                            ((best_of == 5) & (row["Set2"] == 2))):
                            # Player2 has already won 1 or 2 set and now has a set point
                            point_data.at[index, "server_critical_point"] += ",match_point"
                        
            elif row["Pts"] in break_point_scores:
                # Returner has a break point
                point_data.at[index, "returner_critical_point"] = "break_point"
                
                if row["Svr"] == 1:
                    # Server is player1, and player2 has a breakpoint
                    # In Wimbledon and Roland Garros, a player needs to win two games in a row
                    # after 5-5 to win the 5th set. 
                    if (((row["Gm2"] == 5) & (row["Gm1"] <= 4)) | 
                        ((row["Gm2"] > 5) & (row["Gm2"] > row["Gm1"]))):
                        point_data.at[index, "returner_critical_point"] += ",set_point"
                        
                        if (((best_of == 3) & (row["Set2"] == 1)) | 
                            ((best_of == 5) & (row["Set2"] == 2))):
                            # Player2 has already won 1 or 2 set and now has a set point
                            point_data.at[index, "returner_critical_point"] += ",match_point"                        
                else:
                    # Server is player2, and player1 has a break point
                    if (((row["Gm1"] == 5) & (row["Gm2"] <= 4)) | 
                        ((row["Gm1"] > 5) & (row["Gm1"] > row["Gm2"]))):
                        point_data.at[index, "returner_critical_point"] += ",set_point" 

                        if (((best_of == 3) & (row["Set1"] == 1)) | 
                            ((best_of == 5) & (row["Set1"] == 2))):
                            # Player1 has already won 1 or 2 set and now has a set point
                            point_data.at[index, "returner_critical_point"] += ",match_point"
            
            if row["Pts"] in server_setup_point_scores:
                point_data.at[index, "server_critical_point"] = "setup_point"
            
            if row["Pts"] in returner_setup_point_scores:
                point_data.at[index, "returner_critical_point"] = "setup_point" 
            
            if row["Pts"] in deuce_point_score:
                point_data.at[index, "server_critical_point"] = "deuce_point"
                point_data.at[index, "returner_critical_point"] = "deuce_point"
            
        else:
            #Tiebreak points
            tiebreak_point = row["Pts"].split("-")
            
            if len(tiebreak_point) != 2:
                print("wrong tiebreak point" + str(tiebreak_point))
            
            # The original data table already switched score sequence when the server is switched.
            server_point = int(tiebreak_point[0])
            returner_point = int(tiebreak_point[1])
            
            # For AO, if the 5th set is 6-6, they will play a 10 point tiebreak, not 7.
            if ((((tournament == "australian_open") & 
                 (row["Set1"] == 2) & (row["Set2"] == 2)) & 
                 (((server_point == 9) & (returner_point <= 8)) |
                ((server_point > 9) & (server_point > returner_point)))) | 
                (((server_point == 6) & (returner_point <= 5)) | 
               ((server_point > 6) & (server_point > returner_point)))):
                # A tiebreak point is also a set point
                point_data.at[index, "server_critical_point"] = "tiebreak_point,set_point"
                
                # French Open does not have a tiebreak in the 5th set, therefore the 
                # code below does not apply to French Open.
                
                # Since 2019, Wimbledon has a tiebreak after 12-12 in the 5th set. 
                # Therefore, the following code applies to Wimbledon from 2019, if
                # the players reach 12-12.
                
                if row["Svr"] == 1:
                    # Server is player1
                    if (((best_of == 3) & (row["Set1"] == 1)) | 
                        ((best_of == 5) & (row["Set1"] == 2))):
                        # Player1 has already won 1 or 2 set and now has a set point
                        point_data.at[index, "server_critical_point"] += ",match_point"
                else:                        
                    # Server is player2
                    if (((best_of == 3) & (row["Set2"] == 1)) | 
                        ((best_of == 5) & (row["Set2"] == 2))):
                        # Player2 has already won 1 or 2 set and now has a set point
                        point_data.at[index, "server_critical_point"] += ",match_point"                
                
            elif ((((tournament == "australian_open") & 
                 (row["Set1"] == 2) & (row["Set2"] == 2)) & 
                 (((returner_point == 9) & (server_point <= 8)) |
                ((returner_point > 9) & (returner_point > server_point)))) | 
                  (((returner_point == 6) & (server_point <= 5)) | 
               ((returner_point > 6) & (returner_point > server_point)))):
                point_data.at[index, "returner_critical_point"] = "tiebreak_point,set_point"
                
                if row["Svr"] == 1:
                    # Server is player1, and player2 has a tiebreak point 
                    if (((best_of == 3) & (row["Set2"] == 1)) | 
                        ((best_of == 5) & (row["Set2"] == 2))):
                        # Player2 has already won 1 or 2 set and now has a set point
                        point_data.at[index, "returner_critical_point"] += ",match_point"                        
                else:
                    # Server is player2, and player1 has a break point
                    if (((best_of == 3) & (row["Set1"] == 1)) | 
                        ((best_of == 5) & (row["Set1"] == 2))):
                        # Player1 has already won 1 or 2 set and now has a set point
                        point_data.at[index, "returner_critical_point"] += ",match_point"
            
            
            if ((tournament == "austalian_open") & 
               (row["Set1"] == 2) & (row["Set2"] == 2)):
                #For 5th set tiebreak in AO, the setup point is 8-x
                if ((server_point == 8) & (server_point >= returner_point)): 
                    point_data.at[index, "server_critical_point"] = "tb_setup_point"
                
                if ((returner_point == 8) & (returner_point >= server_point)):
                    point_data.at[index, "returner_critical_point"] = "tb_setup_point"
                
                if (server_point == returner_point) & (server_point > 8):
                    point_data.at[index, "server_critical_point"] = "tb_deuce_point"
                    point_data.at[index, "returner_critical_point"] = "tb_deuce_point"
            else:
                # For all the other tournaments
                if (server_point == 5) & (server_point >= returner_point): 
                    point_data.at[index, "server_critical_point"] = "tb_setup_point"

                if (returner_point == 5) & (returner_point >= server_point):
                    point_data.at[index, "returner_critical_point"] = "tb_setup_point"

                if (server_point == returner_point) & (server_point > 5):
                    point_data.at[index, "server_critical_point"] = "tb_deuce_point"
                    point_data.at[index, "returner_critical_point"] = "tb_deuce_point"                
                
        
    return point_data

In [57]:
# Input: point_data (all the point data), match_data (user selected matches)
# Output: df_points_selected (all the points from the selected matches)
# This function is used to select points before analyzing them. 
def get_points_by_matches(point_data, match_data):

    # Use a copy to avoid "A value is trying to be set on a copy of a slice from a DataFrame" warning. 
    point_data = point_data.copy()
    
    # There may be multiple match_ids
    df_selected_points = point_data.loc[point_data["match_id"].isin(match_data["match_id"])]

    # df_points_selected.info()
    return df_selected_points

In [97]:
# Calculate the gaps in point scores, game scores, and set scores. 
# These gaps are used in calculating anxiety indices.
# They may also be used in storytelling analysis.

def calculate_score_gaps(point_data, match_data, point_gap_file): 
    # Everything needs to be normalized to [0:1]
    # Uncertainty = min(server_points_to_win, server_points_to_loss) * server_returner_gap * weightUncertainty
    
    # Hope = (server_points_to_win * server_returner_gap * (serve game * service_game_confidence or return game * return_game_confidence) 
    # * (point_confidence) * critical_moment * weightHope
    
    # Fear = (server_points_to_loss * server_returner_gap * (serve game * service_game_confidence or return game * return_game_confidence) 
    # * (point_confidence) * critical_moment * weightFear
    
    point_data = point_data.copy()
    
    # To-do: add normalized gaps to the point gap table
    
    # The point score gaps for non-tiebreak points are stored in this table 
    # so we don't need to calculate it. 
    # Other cases are too complicated to precalcualte, so we'll have to write code. 
    point_gap_table = pd.read_csv(point_gap_file)
    
#     point_data[["server_points_to_win", "server_points_to_loss", 
#                 "returner_points_to_win", "returner_points_to_loss", 
#                 "server_returner_points_gap", "returner_server_points_gap",
#                 "server_points_to_win_norm", "server_points_to_loss_norm", 
#                 "returner_points_to_win_norm", "returner_points_to_loss_norm", 
#                 "server_returner_points_gap_norm", "returner_server_points_gap_norm"]] = None
    
    # Calculate point score gaps for non-tiebreak points using the pint_gap_table.
    
    # Left merge point_gap_table with the point data based on "Pts".
    # For example, if Pts is "40-30", values in "server_points_to_win", 
    # "returner_points_to_loss," etc. in point_gap_table will fill the 
    # corresponding columns in the point_data.
    # This is the best way to merge two tables with matching columns. 
    point_data = pd.merge(point_data, point_gap_table,
                        how="left", on=["Pts"])
        
    # Calculate point gaps for tiebreak points
    for index, row in point_data.iterrows():        
        if row["TB?"] == 1:
            #Tiebreak points
            tiebreak_point = row["Pts"].split("-")
            
            if len(tiebreak_point) != 2:
                print("wrong tiebreak point" + str(tiebreak_point))
            
            # The original data table already switched score sequence when the server is switched.
            server_point = int(tiebreak_point[0])
            returner_point = int(tiebreak_point[1])

            tb_target_point = 7
            tb_combined_points_for_2_point_gap = 12
            
            # Australian Open's final set tiebreak is a 10 point tiebreak.
            if ((row["tournament"] == "australian_open") & 
                ((row["Set1"] == 2) & (row["Set2"] == 2))):
                tb_target_point = 10
                tb_combined_points_for_2_point_gap = 18

            if (server_point + returner_point) < tb_combined_points_for_2_point_gap:
                # Before 6-6
                point_data.at[index, "server_points_to_win"] = tb_target_point - server_point
                point_data.at[index, "server_points_to_loss"] = tb_target_point - returner_point
                point_data.at[index, "returner_points_to_win"] = tb_target_point - returner_point
                point_data.at[index, "returner_points_to_loss"] = tb_target_point - server_point

                point_data.at[index, "server_returner_points_gap"] = server_point - returner_point
                point_data.at[index, "returner_server_points_gap"] = returner_point - server_point
                
                # Calculate normalized gaps (between 0 and 1) so we can compare the gaps between 
                # tiebreak scores and non-tiebreak scores.
                point_data.at[index, "server_points_to_win_norm"] = \
                    point_data.at[index, "server_points_to_win"] / tb_target_point
            
                point_data.at[index, "server_points_to_loss_norm"] = \
                     point_data.at[index, "server_points_to_loss"] / tb_target_point

                point_data.at[index, "returner_points_to_win_norm"] = \
                    point_data.at[index, "returner_points_to_win"] / tb_target_point

                point_data.at[index, "returner_points_to_loss_norm"] = \
                    point_data.at[index, "returner_points_to_loss"] / tb_target_point

                # The biggest possible point gap in tiebreak is 6 (or 9 in AO final set tiebreak)
                point_data.at[index, "server_returner_points_gap_norm"] = \
                    point_data.at[index, "server_returner_points_gap"] / (tb_target_point - 1)

                point_data.at[index, "returner_server_points_gap_norm"] = \
                    point_data.at[index, "returner_server_points_gap"] / (tb_target_point - 1)
                
            else: 
                # After 6-6 (or 9-9 for AO final et)
                point_data.at[index, "server_points_to_win"] = 2 - (server_point - returner_point)
                point_data.at[index, "server_points_to_loss"] = 2 - (returner_point - server_point)
                point_data.at[index, "returner_points_to_win"] = 2 - (returner_point - server_point)
                point_data.at[index, "returner_points_to_loss"] = 2 - (server_point - returner_point)
                
                point_data.at[index, "server_returner_points_gap"] = server_point - returner_point
                point_data.at[index, "returner_server_points_gap"] = returner_point - server_point                
                            
                # Calculate normalized gaps so we can compare the gaps between tiebreak scores and 
                # non-tiebreak scores.
                # When the tiebreak scores are tied at 6, each sides needs to win two points in
                # a row to win the tiebreak. Now the most point one can win is 3 points. 
                # The biggest possible gap is 1. 
                point_data.at[index, "server_points_to_win_norm"] = \
                    point_data.at[index, "server_points_to_win"] / 3
            
                point_data.at[index, "server_points_to_loss_norm"] = \
                     point_data.at[index, "server_points_to_loss"] / 3

                point_data.at[index, "returner_points_to_win_norm"] = \
                    point_data.at[index, "returner_points_to_win"] / 3

                point_data.at[index, "returner_points_to_loss_norm"] = \
                    point_data.at[index, "returner_points_to_loss"] / 3

                point_data.at[index, "server_returner_points_gap_norm"] = \
                    point_data.at[index, "server_returner_points_gap"] 

                point_data.at[index, "returner_server_points_gap_norm"] = \
                    point_data.at[index, "returner_server_points_gap"] 
        
    # Calculate game score gap

    point_data[["server_games_to_win", "server_games_to_loss", 
                "returner_games_to_win", "returner_games_to_loss", 
                "server_returner_games_gap", "returner_server_games_gap",
                "server_games_to_win_norm", "server_games_to_loss_norm", 
                "returner_games_to_win_norm", "returner_games_to_loss_norm", 
                "server_returner_games_gap_norm", "returner_server_games_gap_norm"]] = None
    
    for index, row in point_data.iterrows():        
        if row["TB?"] == 0:
            # Non-tiebreak points
            
            if row["Svr"] == 1:
                server_game_score = int(row["Gm1"])
                returner_game_score = int(row["Gm2"])
            else:
                server_game_score = int(row["Gm2"])
                returner_game_score = int(row["Gm1"])
                
            if (server_game_score + returner_game_score) <= 9:
                # Before the game scores are tied at 5-5
                point_data.at[index, "server_games_to_win"] = 6 - server_game_score
                point_data.at[index, "server_games_to_loss"] = 6 - returner_game_score
                point_data.at[index, "returner_games_to_win"] = 6 - returner_game_score
                point_data.at[index, "returner_games_to_loss"] = 6 - server_game_score
                
                # Before 5-5, biggest possible games to win or loss is 6
                point_data.at[index, "server_games_to_win_norm"] = \
                    point_data.at[index, "server_games_to_win"] / 6
                
                point_data.at[index, "server_games_to_loss_norm"] = \
                    point_data.at[index, "server_games_to_loss"] / 6
                    
                point_data.at[index, "returner_games_to_win_norm"] = \
                    point_data.at[index, "returner_games_to_win"] / 6
                
                point_data.at[index, "returner_games_to_loss_norm"] = \
                    point_data.at[index, "returner_games_to_loss"] / 6
                
            else: 
                # After the game sctores are tied after 5-5
                
                point_data.at[index, "server_games_to_win"] = 7 - server_game_score
                point_data.at[index, "server_games_to_loss"] = 7 - returner_game_score
                point_data.at[index, "returner_games_to_win"] = 7 - returner_game_score
                point_data.at[index, "returner_games_to_loss"] = 7 - server_game_score
                
                # The most games you need to win or lose are 2. 
                # divide by 2
                
                point_data.at[index, "server_games_to_win_norm"] = \
                    point_data.at[index, "server_games_to_win"] / 2
                
                point_data.at[index, "server_games_to_loss_norm"] = \
                    point_data.at[index, "server_games_to_loss"] / 2 
                
                point_data.at[index, "returner_games_to_win_norm"] = \
                    point_data.at[index, "returner_games_to_win"] / 2
                
                point_data.at[index, "returner_games_to_loss_norm"] = \
                    point_data.at[index, "returner_games_to_loss"] / 2
                
                # If it's the fifth set of french open or wimbledon, then override the above.
                # In these two tournaments, there is no tiebreak in the 5th. 
                # To-do: Wimbledon introduced a tiebreak in the 5th set after 12-12.
                # The current code does not deal with it yet. 
                tournament = match_data.loc[match_data["match_id"] == row["match_id"], "Tournament"]
                tournament = tournament.iloc[0]
        
                if ((tournament in ["roland_garros", "wimbledon"]) & 
                    ((row["Set1"] == 2) & (row["Set2"] == 2))):
                    point_data.at[index, "server_games_to_win"] = \
                        (2 - (server_game_score - returner_game_score))
                    point_data.at[index, "server_games_to_loss"] = \
                        (2 - (returner_game_score - server_game_score))
                    point_data.at[index, "returner_games_to_win"] = \
                        (2 - (returner_game_score - server_game_score))
                    point_data.at[index, "returner_games_to_loss"] = \
                        (2 - (server_game_score - returner_game_score))
                    
                    # In french open and wimbledon, the most games you need to win or lose are 3. 
                    # Divide by 3
                    point_data.at[index, "server_games_to_win_norm"] = \
                        point_data.at[index, "server_games_to_win"] / 3
                
                    point_data.at[index, "server_games_to_loss_norm"] = \
                        point_data.at[index, "server_games_to_loss"] / 3 

                    point_data.at[index, "returner_games_to_win_norm"] = \
                        point_data.at[index, "returner_games_to_win"] / 3

                    point_data.at[index, "returner_games_to_loss_norm"] = \
                        point_data.at[index, "returner_games_to_loss"] / 3
                
            point_data.at[index, "server_returner_games_gap"] = (server_game_score - 
                                                                returner_game_score)
            point_data.at[index, "returner_server_games_gap"] = (returner_game_score - 
                                                                 server_game_score)
            
            # For non-tiebreak points, the largest possible gap is 6.
            point_data.at[index, "server_returner_games_gap_norm"] = \
                point_data.at[index, "server_returner_games_gap"] / 6
            point_data.at[index, "returner_server_games_gap_norm"] = \
                point_data.at[index, "returner_server_games_gap"] / 6
        else: 
            # tiebreak points
            point_data.at[index, "server_games_to_win"] = 1
            point_data.at[index, "server_games_to_loss"] = 1
            point_data.at[index, "returner_games_to_win"] = 1
            point_data.at[index, "returner_games_to_loss"] = 1
            point_data.at[index, "server_returner_games_gap"] = 0
            point_data.at[index, "returner_server_games_gap"] = 0
            
            # In a tiebreak, the most game you can win or loss is always 1.
            point_data.at[index, "server_games_to_win_norm"] = \
                point_data.at[index, "server_games_to_win"]
            
            point_data.at[index, "server_games_to_loss_norm"] = \
                point_data.at[index, "server_games_to_loss"]
            
            point_data.at[index, "returner_games_to_win_norm"] = \
                point_data.at[index, "returner_games_to_win"]
            
            point_data.at[index, "returner_games_to_loss_norm"] = \
                point_data.at[index, "returner_games_to_loss"]
            
            # For tiebreak points, the game_points_gap is 0.
            point_data.at[index, "server_returner_games_gap_norm"] = \
                point_data.at[index, "server_returner_games_gap"]
            
            point_data.at[index, "returner_server_games_gap_norm"] = \
                point_data.at[index, "returner_server_games_gap"]            
    
    # Calculate set point gap
    point_data[["server_sets_to_win", "server_sets_to_loss", 
                "returner_sets_to_win", "returner_sets_to_loss", 
                "server_returner_sets_gap", "returner_server_sets_gap",
                "server_sets_to_win_norm", "server_sets_to_loss_norm", 
                "returner_sets_to_win_norm", "returner_sets_to_loss_norm", 
                "server_returner_sets_gap_norm", "returner_server_sets_gap_norm"]] = None   
    
    for index, row in point_data.iterrows():
        
        best_of = match_data.loc[match_data["match_id"] == row["match_id"], "Best of"]
        best_of = int(best_of.iloc[0])
    
        if row["Svr"] == 1:
            server_set_score = int(row["Set1"])
            returner_set_score = int(row["Set2"])
        else:
            server_set_score = int(row["Set2"])
            returner_set_score = int(row["Set1"])
        
        if best_of == 5:
            target_sets = 3
            max_set_score_gap = 2
        elif best_of == 3:
            target_sets = 2
            max_set_score_gap = 1
            
        point_data.at[index, "server_sets_to_win"] = target_sets - server_set_score
        point_data.at[index, "server_sets_to_loss"] = target_sets - returner_set_score
        point_data.at[index, "returner_sets_to_win"] = target_sets - returner_set_score
        point_data.at[index, "returner_sets_to_loss"] = target_sets - server_set_score
        
        # Normalize the value of gaps so they are comparable with other gaps
        # We can then use them in the same equation.
        point_data.at[index, "server_sets_to_win_norm"] = \
            point_data.at[index, "server_sets_to_win"] / target_sets
        point_data.at[index, "server_sets_to_loss_norm"] = \
            point_data.at[index, "server_sets_to_loss"] / target_sets
        point_data.at[index, "returner_sets_to_win_norm"] = \
            point_data.at[index, "returner_sets_to_win"] / target_sets
        point_data.at[index, "returner_sets_to_loss_norm"] = \
            point_data.at[index, "returner_sets_to_loss"] / target_sets
        
        point_data.at[index, "server_returner_sets_gap"] = server_set_score - returner_set_score
        point_data.at[index, "returner_server_sets_gap"] = returner_set_score - server_set_score
        
        # devide by max_set_points_gap
        point_data.at[index, "server_returner_sets_gap_norm"] = \
            point_data.at[index, "server_returner_sets_gap"] / max_set_score_gap
        point_data.at[index, "returner_server_sets_gap_norm"] = \
            point_data.at[index, "returner_server_sets_gap"] / max_set_score_gap
    
    # For game anxiety, in addition to win_gap, loss_gap, and server_returner_game_gap, 
    # also consider (num_service_games - num_return_games). If you have more service games, 
    # then you are less anxious. 
    # Should we consider form? How do we calculate form? Confidence is a measure of form. 
    
    return point_data

In [25]:
# To-do
# Can specify same serve direction to get the outcome of the last service point 
# with the same direction and same serve side
def get_last_service_point():
    
# Can specify the outcome of the last return point from the same side.
def get_last_return_point():
    
def get_last_point():

def get_last_long_rally():
    
# Get the numbero of unforced errors in the last X points
def get_num_prev_unforced_errors(): 
    
def get_num_prev_winners():
    
def get_num_prev_forced_errors():

# Get the number of first serve faults for the last X service points
# Can specify serve direction 
def get_num_prev_first_serve_faults():
    
def get_num_prev_double_faults():
    
def get_num_prev_return_errors():
    
def calc_fatigue_index():
    
def calc_run_index(): 
    
def calc_aggressiveness_index():

def calc_variety_index():

def identify_comeback():
    
def identify_reversal():
    
def identify_new_ball(): 

def get_num_unforced_errors_current_game()

def get_num_winners_current_game()

def get_num_forced_errors_current_game()

# To calculate confidence level 
def get_num_unforced_errors(current_game, last_points)

# Find style, find favorite pattern, can you group them? 

IndentationError: expected an indented block (<ipython-input-25-a26019552785>, line 7)

## Commonly used functions end here. 

Data cleaning and pre-processing start below.

## 3. Load raw match data and perform data cleaning and pre-processing

If the raw match data is not changed and you have processed match data, skip this step. 


In [61]:
# You may need to manually correct the CSV file. On line 955, there is a cell where there is a starting "
# but without the closing ". Need to delete the " mark.

#Use xls, not .csv or .xlsx, because they usually cause errors.
df_matches = pd.read_excel("charting-m-matches.xls")

In [62]:
df_matches = clean_match_data(df_matches)

In [63]:
df_matches, df_players_handedness = clean_handedness_match_data(df_matches)

stanislas_wawrinka
r    20
l     1
Name: Pl 2 hand, dtype: int64
rafael_nadal
l    87
r     1
Name: Pl 1 hand, dtype: int64
rafael_nadal
l    160
r      1
Name: Pl 2 hand, dtype: int64
kyle_edmund
r    2
l    1
Name: Pl 2 hand, dtype: int64
nick_kyrgios
r    8
l    1
Name: Pl 1 hand, dtype: int64
thomas_muster
l    11
r     1
Name: Pl 2 hand, dtype: int64


In [30]:
df_players_handedness.to_csv("player_handedness.csv", index=False)

## The match data still needs to be synchronized with the point data. So the match data is not ready yet. 

## 4. Load raw point data and perform data cleaning and pre-processing

If the raw point data is not changed and you have processed point data, skip this step and load the processed data directly. 

Start here when you download a new charting-m-points.csv file from GitHub.

Otherwise, skip this part and start from the cell that loads the processed data. 

The cell below may take a long time to run. 

In [31]:
# This is the "CSV UTF-8" file.

# Use "charting-m-points_1.csv" if not testing
df_points = pd.read_csv("charting-m-points_1.csv", low_memory=False)

df_points = clean_point_data(df_points) 

In [ ]:
# This can take a long time. 

# Add all the new data processing functions here.

%time df_points = identify_serve_side(df_points)
print("finished identifying serve side")                    

%time df_points = identify_serve_direction_outcome(df_points)
print("finished identifying serve direction and outcome")

%time df_points = identify_server_name(df_points)
print("finished identifying serve direction and outcome")

# Save the results so we don't need to do this every time.
df_points.to_csv("charting-m-points_with_serve_side.csv")

## 5. Load the processed point data

Start here if you already have processed data. It's much faster.

In [64]:
# Read the processed data
# This file contains the serve side, serve outcome, server, and returner for each point. 
df_points = pd.read_csv("charting-m-points_with_serve_side.csv", low_memory=False)

df_points = clean_point_data(df_points)

### Only run the cell below if the match data is created from the original raw data.
### Adding missing match_ids and data from the point data to the match data.
### If the match data is already synced with the point data, you can skip to 6 (load processed match data directly). 

In [65]:
df_matches = sync_match_and_point_data(df_points, df_matches)

df_matches = clean_match_data(df_matches)

Checking if points data and match data are consistent ... 
There are 2350 match ids in the match data set.
There are 2573 match ids in the point data set.
The number of match ids do not match. Trying to fix the problem ...
The following match ids are in the match data set but not the point data set. Fix it.

Match data and point data are the same at 2573
Problem solved


## After processing and merging the point and match data, save it to a CSV file.

In [66]:
df_matches.to_csv("match_data_cleaned.csv", index=False)

## 6. Load processed match data

Start here if you already have processed data.

In [38]:
import pandas as pd

df_matches = pd.read_csv("match_data_cleaned.csv")

### Get information about the match data

In [67]:
players = get_players(df_matches)

len(players)

550

In [68]:
player_match_count = get_player_match_count(df_matches)

player_match_count

roger_federer      447
novak_djokovic     275
rafael_nadal       267
andy_murray        153
stefan_edberg      130
                  ... 
markus_eriksson      1
zhizhen_zhang        1
mate_delic           1
vincent_millot       1
vaclav_safranek      1
Length: 550, dtype: int64

In [160]:
handedness_count = get_handedness_count(df_players_handedness)

handedness_count

r    442
l     75
Name: handedness, dtype: int64

In [161]:
match_count_by_tourney = get_match_count_by_tournament(df_matches)

match_count_by_tourney

australian_open         210
wimbledon               190
us_open                 175
roland_garros           126
indian_wells_masters    121
                       ... 
biella_ch                 1
chengdu                   1
drummondville_ch          1
orleans_ch                1
philadelphia              1
Name: Tournament, Length: 241, dtype: int64

In [164]:
player_match_count_by_tournament = get_player_match_count_by_tournament(df_matches)

player_match_count_by_tournament

C:\Users\yingz\Anaconda3\lib\site-packages\pandas\core\frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Player,Tournament,Year,Count
3201,roger_federer,australian_open,2018,7
3406,roger_federer,us_open,2015,7
3006,rafael_nadal,roland_garros,2014,7
3300,roger_federer,indian_wells_masters,2018,6
2624,novak_djokovic,olympics,2012,6
...,...,...,...,...
1458,james_blake,cincinnati_masters,2007,1
1460,james_blake,masters_cup,2006,1
1461,james_blake,olympics,2008,1
1462,james_blake,queens_club,2006,1


In [166]:
h2h_count = get_player_head_to_head_count(df_matches)

h2h_count

,Player 1,Player 2,Count
2259,novak_djokovic,rafael_nadal,48
2501,rafael_nadal,novak_djokovic,48
2751,roger_federer,novak_djokovic,47
2263,novak_djokovic,roger_federer,47
2760,roger_federer,rafael_nadal,35
...,...,...,...
1163,hubert_hurkacz,novak_djokovic,1
1164,hubert_hurkacz,roger_federer,1
1166,hubert_hurkacz,taylor_harry_fritz,1
1167,hugo_dellien,aljaz_bedene,1


In [167]:
tournament_count = get_tournament_count(df_matches)

tournament_count

,Tournament,Year,Count
993,wimbledon,2019,39
401,indian_wells_masters,2019,32
522,miami_masters,2019,25
66,australian_open,2019,22
67,australian_open,2020,22
...,...,...,...
432,long_island,1990,1
434,los_angeles,1999,1
435,los_angeles,2001,1
436,los_angeles,2008,1


In [168]:
# Find the number of matches for a player 
player_name = "federer"
tournament = "us_open"

selected_matches = get_matches(df_matches, player1=player_name, tournament=tournament)
print(len(selected_matches))
selected_matches.info

28


<bound method DataFrame.info of                                                match_id            Player 1  \
636   20180903-M-US_Open-R16-Roger_Federer-John_Millman       roger_federer   
638   20180901-M-US_Open-R32-Roger_Federer-Nick_Kyrgios       roger_federer   
642   20180830-M-US_Open-R64-Benoit_Paire-Roger_Federer        benoit_paire   
644   20180828-M-US_Open-R128-Yoshihito_Nishioka-Rog...  yoshihito_nishioka   
1370  20110909-M-US_Open-SF-Roger_Federer-Novak_Djok...       roger_federer   
1426  20100903-M-US_Open-SF-Novak_Djokovic-Roger_Fed...      novak_djokovic   
1472  20090914-M-US_Open-F-Roger_Federer-Juan_Martin...       roger_federer   
1473  20090913-M-US_Open-SF-Novak_Djokovic-Roger_Fed...      novak_djokovic   
1476  20090905-M-US_Open-R32-Roger_Federer-Lleyton_H...       roger_federer   
1519     20080908-M-US_Open-F-Roger_Federer-Andy_Murray       roger_federer   
1572  20070909-M-US_Open-F-Roger_Federer-Novak_Djokovic       roger_federer   
1573  20070908-M-US_

In [169]:
# Find the head-to-head match count for two players
player1_name = "nadal"
player2_name = "federer"

len(get_matches(df_matches, player1=player1_name, player2=player2_name))

35

## For testing new point data processing functions

### Because it takes too long to process the entire point data table, I will select one match and test the new point data processing functions. The goal is to apply these functions to the entire point data table. 

If they work well, I will copy these functions to the cells above to apply them to the entire point data table. 

In [99]:
# Select one match and get the points
# No need to do this every time. We only need to run this once and save it to a CSV file.
# After that, we just load the CSV file.
my_matches = get_matches(df_matches, player1 = "federer", player2 = "nadal", date="2008", tournament = "wimbledon")

test_points = get_points_by_matches(df_points, my_matches)

test_points = clean_point_data(test_points)

# Test new point data processing functions. Some data processing functions have been run before. 
%time test_points = identify_server_returner_point_winner(test_points)
%time test_points = separate_server_returner_shots(test_points)
%time test_points = identify_critical_points(test_points, df_matches)
%time test_points = calculate_score_gaps(test_points, df_matches, "point_gap_table.csv")

test_points.sort_values(by=["match_id", "Pt"], inplace = True)

# test_points = test_points[["Set1", "Set2", "Gm1", "Gm2", "Pts", "server_critical_point", "returner_critical_point"]]

test_points.to_csv("test_points_processed.csv", index=False)

Wall time: 292 ms
Wall time: 65.3 ms
Wall time: 367 ms
Wall time: 455 ms




Add specific data analysis and visualization code below. 

### To-do:
- Identify point outcome (done)
- Separate the shot sequence for each player (done)
- Calculate fatigue index for each point and the cumulative fatigue index for each player and each point
- Calculate run index for each point and the cumulative run index for each player and each point
- Write a function to get the outcome of the last X number of points with rallies. Maybe differentiate between short and long rallies. This can be used to calculate rally confidence. 
- Write a function to get the outcome of the last point with the same serve direction. 
- Write a function to get the outcome of the last X number of serves faults. This can be used to calculate serve confidence. 
- Write a function to get the outcome of the last X number of returns. This can be used to calcualte return confidence. 
- Write a function to calculate the fear, hope, and uncertainty scores for each player and each point. 
- Write a function to identify crucial points. How to define crucial points? Two points from game (1), one-point from game (2), set point (add weight) (3), match point (add weight) (4). Breakpoint should have a higher weight.  
- Write a function to calculate the residue emotions after each game and each set (relief, excitement, frustration, anger)
- Calculate the gap between winning, losing, and the gap between the two players
- Write a function to evaluate the aggressiveness of a rally for a player
- Detect patterns in a player's rally. Can you classify players into "style" groups?
- Write a function to convert from traditional tennis score to point score
- Add "server first serve streak", "server second serve streak", "server/returner point streak", 
- How to evaluate a rally? Find statistical parameters for a rally:
    - Number of different shot types per point (how to measure variety?)
    - Distribution1a of different shot types 
    - Length of rally
    - Aggressiveness
    - Run index
- Long rally winning percentage?

## 3. Analyzing serve patterns

In [1]:
from scipy.stats import chi2_contingency
from scipy.stats import chi2

# Chi-square independence test
def chi_square(data, print_table = False):

    print("Chi-square Independence Test")
    table = data.copy()

#     table = table.set_index("server")

    test_stats, p, dof, expected = chi2_contingency(table)
#     print("chi-squared test results:")
#     print("chi2 test stats = %f" % test_stats)
#     print("p = %f" % p)
#     print('dof = %d ' % dof)

     # interpret test-statistic
    prob = 0.95
    critical = chi2.ppf(prob, dof)
    print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, test_stats))
#     if abs(test_stats) >= critical:
#         print('Dependent (reject H0)')
#     else:
#         print('Independent (fail to reject H0)')

     # interpret p-value
#     alpha = 1.0 - prob
#     print('significance=%.3f, p=%.3f' % (alpha, p))
#     if p <= alpha:
#         print('Dependent (reject H0)')
#     else:
#         print('Independent (fail to reject H0)')

    if print_table:
        print(table)
        
    return p

In [ ]:
# May need the commented code for windows
# from __main__ import *
# import os
# os.environ['PYTHONHOME'] = 'C:/Program Files/Python'
# os.environ['PYTHONPATH'] = 'C:/Program Files/Python/lib/site-packages'
# os.environ['R_HOME'] = 'C:\\Program Files\\R\\R-4.0.2'
# os.environ['R_USER'] = 'C:/Program Files/Python/Lib/site-packages/rpy2'

# I can't find a good implementation of Freeman-Halton Test in Python.
# Has to use R via rpy2 package. 
# rpy2 works well on Mac, but doesn't seem to work well on Windows 10. 
import numpy as np
import rpy2.robjects.numpy2ri
from rpy2.robjects.packages import importr
rpy2.robjects.numpy2ri.activate()

# Use the "stats" package in R. 
stats = importr('stats')

# Freeman-Halton test is an extension of the Fisher's Exact Test. 
# It can handle 2x3 tables, which is needed for this project. 
# It's suitable for small sample size.
# data is numpy ndarray
def freeman_halton_test(data):
#     print("Freeman-Halton Test")
#     m = np.array([[4,4],[4,5],[10,6]])
    res = stats.fisher_test(data)
    # print('p-value: {}'.format(res[0][0]))
    p_value = res[0][0]
#     print("p = " + str(p_value))
    
#     if p_value <= 0.05:
#         print("Dependent (reject H0)")
#     else:
#         print("Independent (fail to reject H0)")
        
    return p_value

In [ ]:
from scipy.stats import chisquare
def serve_direction_test_of_even_distribution(point_data, match_data, player, opponent=None):
    surfaces = ["hard", "clay", "grass"]
    serve_sides = ["ad", "deuce"]
    
    serve_sequences = ["Sv1_direction", "Sv2_direction"]
        
    contingency_table = pd.DataFrame()
    
    for serve_sequence in serve_sequences:
        print(serve_sequence)
        for serve_side in serve_sides:
            print(serve_side)
            for surface in surfaces:
                print(surface)
                matches_by_surface = select_matches(data=match_data, player1=player, player2=opponent, surface=surface)
                selected_points = select_players_points(point_data, matches_by_surface)
                print("finished selecting points")

                if selected_points.empty != True:
                    my_data = selected_points.loc[(selected_points["server_name"].str.contains(player))]
                    # Without Series, you cannot use value_counts()
                    serve_frequency = my_data.loc[my_data["serve_side"] == serve_side, serve_sequence].value_counts()
                    serve_frequency.drop(labels="unknown", errors="ignore", inplace=True)
                    # Replace nan with 0
#                     serve_frequency.fillna(value=0, inplace=True)
                    # Sometimes the serve direction is not entered and is marked as nan, remove these counts. 
                    
                    serve_frequency_list = serve_frequency.tolist()
                    print(serve_frequency)
                    chisq, p = chisquare(serve_frequency_list)
                    print("chi-square: " + str(chisq))
                    print("p: " + str(p))
                    if p < 0.05:
                        print("Reject null hypothesis, not evenly distributed.\n")
                    else:
                        print("Cannot reject null hypothesis, possibly evenly distributed.\n")
                else:
                    print("No matches found")


In [ ]:
#some players may generate errors due to NaN existing in the "Surface" column
serve_direction_test_of_even_distribution(point_data=df_points, match_data=df_matches, player="nadal", opponent=None)
# nadal_matches = select_matches(data=df_matches, player1="nadal")
# nadal_matches["Surface"].value_counts(dropna=False)
# possible even distribution:
# goffin, shapovalov, tsitsipas, wawrinka, pouille

## To-do

Within-subject reliability/consistency test:
- Internal consistency: whether the serve pattern is consistent within a match (very short time -- hours)
- External consistency: whether the serve pattern is consistent over time in a tournament (short time -- days)
- External consistency: Check if a player's serve pattern is consistent against the same player on the same surface or at the same tournament, year after year (long time)
- Check if the serve direction patterns vary from match to match on the same surface and same serve side (use MANOVA)

Within-subject differences:
- Compare serve patterns on ad and deuce side in the same match
- Compare serve patterns against different opponents in the same tournament
- Compare serve patterns against the same player on different surfaces/tournaments

Between-subject differences:
- Compare two players' serve patterns when they play each other
- Compare the serve patterns of different players against the same player in the same tournament

## Check if there is a difference in serve patterns between ad and deuce sides

In [ ]:
# Input: player, match_data
# Output: a table with the following columns: player, match_id, surface, opponent, serve_sequence, p_value

def compare_players_serves_on_different_sides(player, first_serve=True, surface=None, opponent=None):
    selected_matches = select_matches(data=df_matches, player1=player, player2=opponent, surface=surface)
    selected_points = select_players_points(df_points, selected_matches)
    
    if first_serve == True:
        serve_sequence = "Sv1_direction"
    else:
        serve_sequence = "Sv2_direction"
        
    if selected_points.empty != True:
        my_data = selected_points.loc[(selected_points["server_name"].str.contains(player))]
        contingency_table = pd.DataFrame()
            
        for serve_side in my_data["serve_side"].unique():
            # Must use .loc[], as not using .loc[] does not result in a Series
            #Without Series, you cannot use value_counts()
            serve_frequency = my_data.loc[(my_data["serve_side"] == serve_side), serve_sequence].value_counts()
            serve_frequency.drop(labels="unknown", errors="ignore", inplace=True)
            serve_frequency["serve_side"] = serve_side
            contingency_table = contingency_table.append(serve_frequency, ignore_index = True)
        contingency_table = contingency_table.dropna(how="all")
        contingency_table = contingency_table.set_index("serve_side")
#         contingency_table.fillna(value=0, inplace=True)
        contingency_table = contingency_table[["wide", "body", "t"]]
        print(contingency_table)
        chi_square(contingency_table, print_table = False)
    else:
        print("No matches found")

In [ ]:
compare_players_serves_on_different_sides("nadal", first_serve=True, surface="clay", opponent=None)

# Check if there is a difference in serve directions on different surfaces
- Need to control by year and/or by opponent

In [ ]:
def compare_players_serves_on_different_surfaces(player, first_serve=True, opponent=None):
    surfaces = ["hard", "clay", "grass"]
    serve_sides = ["ad", "deuce"]
    
    if first_serve == True:
        serve_sequence = "Sv1_direction"
    else:
        serve_sequence = "Sv2_direction"
        
    contingency_table = pd.DataFrame()
    
    for serve_side in serve_sides:
        print(serve_side)
        for surface in surfaces:
            matches_by_surface = select_matches(data=df_matches, player1=player, player2=opponent, surface=surface)
            selected_points = select_players_points(df_points, matches_by_surface)

            if selected_points.empty != True:
                my_data = selected_points.loc[(selected_points["server_name"].str.contains(player))]

                # Without Series, you cannot use value_counts()
                serve_frequency = my_data.loc[my_data["serve_side"] == serve_side, serve_sequence].value_counts()
                serve_frequency.drop(labels="unknown", errors="ignore", inplace=True)
                serve_frequency["surface"] = surface
                contingency_table = contingency_table.append(serve_frequency, ignore_index = True)    
            else:
                print("No matches found")

        contingency_table = contingency_table.dropna(how="all")
        contingency_table = contingency_table.set_index("surface")
        contingency_table.fillna(value=0, inplace=True)
        contingency_table = contingency_table[["wide", "body", "t"]]
        print(contingency_table)
        chi_square(contingency_table, print_table = False)
        contingency_table.drop(contingency_table.index, inplace=True)

In [ ]:
compare_players_serves_on_different_surfaces("murray", first_serve=False)

## Check the difference between two players' serve patterns when they play each other

- To-do: control by surface

In [322]:
def compare_player_serves_when_they_play_each_other(point_data, match_data, player1, player2, test_mode="freeman-halton"):
    
    df_points_selected = select_players_points(point_data=point_data, match_data=match_data)
    
    p_value = None
    
    df_two_player_serve_comparison = pd.DataFrame()
    
    if df_points_selected.empty == False:      
        players = [player1, player2]
        serve_sides = ["deuce", "ad"]
        serve_seqs = ["Sv1_direction", "Sv2_direction"]
        
        for serve_side in serve_sides:
#             print(serve_side)
            for serve_seq in serve_seqs:
#                 print(serve_seq)
                contingency_table = pd.DataFrame()

                for player in players:
                    serve_dir_counts = df_points_selected.loc[(df_points_selected["server_name"].str.contains(player)) & 
                                                              (df_points_selected["serve_side"] == serve_side), serve_seq].value_counts()
        #             print(serve_dir_counts)
                    serve_dir_counts.drop(labels = "unknown", inplace=True, errors="ignore")
                    serve_dir_counts["server_name"] = player
                    contingency_table = contingency_table.append(serve_dir_counts, ignore_index=True)

                contingency_table.set_index("server_name", inplace=True)
                contingency_table.fillna(0, inplace=True)
#                 contingency_table = contingency_table[["wide", "body", "t"]]
                
#                 print(contingency_table)
                
                if (test_mode == "freeman-halton"):
                    p_value = freeman_halton_test(contingency_table.to_numpy())
                
                elif test_mode == "chi-square":
                    # The sample size may be too small for chi-square test. 
                    p_value = chi_square(contingency_table)
                    
                serve_comparison = pd.Series({"player_1": player1, 
                                             "player_2": player2, 
                                             "serve_side": serve_side,
                                             "serve_sequence": serve_seq,
                                             "p_value": p_value})
                
                df_two_player_serve_comparison = df_two_player_serve_comparison.append(serve_comparison, 
                                                                                       ignore_index=True)
                
#                 print(p_value)
    else:
        print("No match found")
        
    return df_two_player_serve_comparison

In [337]:
player_hth_count.head(60)

,Player 1,Player 2,Count
2500,rafael_nadal,novak_djokovic,48
2258,novak_djokovic,rafael_nadal,48
2750,roger_federer,novak_djokovic,47
2262,novak_djokovic,roger_federer,47
2511,rafael_nadal,roger_federer,35
2759,roger_federer,rafael_nadal,35
2206,novak_djokovic,andy_murray,30
307,andy_murray,novak_djokovic,30
2661,roger_federer,andy_murray,24
429,boris_becker,stefan_edberg,24


In [ ]:
# Compare players' serves when they play each other
# player1 = "djokovic"
# player2 = "nadal"
# tournament = None
# surface = "grass"
# date = "2018"

# my_matches = select_matches(df_matches, player1=player1, player2=player2, 
#                             tournament=tournament, surface=surface, date = date)
# print(str(len(my_matches)) + " matches")
# print()

# if (len(my_matches) > 0):
#     df_two_player_serve_comparison = compare_player_serves_when_they_play_each_other(df_points, my_matches, player1, player2, 
#                                                               test_mode="freeman-halton")
#     print(df_two_player_serve_comparison)
# else:
#     print("No match found")
    
df_player_hth_serve_comparison = pd.DataFrame()

for index, row in player_hth_count[player_hth_count["Count"] >= 8].iterrows():
    player1 = row["Player 1"]
    player2 = row["Player 2"]
    player_matches = select_matches(data=df_matches, player1=player1, 
                                    player2=player2)
    print(player1 + " vs " + player2)
    print(len(player_matches))
    
    for match_id in player_matches["match_id"]:
        print(match_id)
        selected_match = df_matches.loc[df_matches["match_id"] == match_id]
        df_two_player_serve_comparison = compare_player_serves_when_they_play_each_other(df_points, 
                                                            selected_match,
                                                            player1, player2, 
                                                            test_mode="freeman-halton")
        if (df_two_player_serve_comparison.empty == False):
            df_two_player_serve_comparison["match_id"] = row_match["match_id"]
            df_player_hth_serve_comparison = \
            df_player_hth_serve_comparison.append(df_two_player_serve_comparison, 
                                                  ignore_index=True)

df_player_hth_serve_comparison.head(20)

In [339]:
df_player_hth_serve_comparison.to_excel("player_hth_serve_comparison.xls")

In [340]:
player_hth_serve_difference = pd.DataFrame()

for index, row in player_hth_count.loc[player_hth_count["Count"] >= 8].iterrows():
    player1 = row["Player 1"]
    player2 = row["Player 2"]
    
    serve_sides = ["deuce", "ad"]
    serve_sequences = ["Sv1_direction", "Sv2_direction"]
    
    for serve_side in serve_sides:
        for serve_seq in serve_sequences:
            selected_stats = df_player_hth_serve_comparison.loc[(df_player_hth_serve_comparison["serve_side"] == serve_side) & 
                                                   (df_player_hth_serve_comparison["serve_sequence"] == serve_seq) & 
                                                   (df_player_hth_serve_comparison["player_1"] == player1) & 
                                                   (df_player_hth_serve_comparison["player_2"] == player2)]
            pct_serve_different = len(selected_stats.loc[selected_stats["p_value"] < 0.05]) / len(selected_stats)
            
            entry = {"player_1": player1,
                    "player_2": player2,
                    "serve_side": serve_side,
                    "serve_sequence": serve_seq,
                    "% of matches with different serve patterns": pct_serve_different}
            
            player_hth_serve_difference = player_hth_serve_difference.append(entry, ignore_index=True)

In [342]:
player_hth_serve_difference.to_excel("player_hth_serve_difference.xls")

In [ ]:
# Compare players' serves when they play the same player on the same surface
def compare_player_serves_when_they_play_same_opponent(point_data, match_data, player1, player2, opponent, surface, test_mode="freeman-halton"):
    
    my_matches1 = select_matches(data=match_data, player1=player1, player2=opponent, surface=surface)
    if my_matches1.empty:
        print("No match found between " + player1 + " and " + opponent)
    else:
        print("%d matches between %s and %s" % (len(my_matches1), player1, opponent))
        
    df_points_selected1 = select_players_points(point_data=point_data, match_data=my_matches1)
    
    my_matches2 = select_matches(data=match_data, player1=player2, player2=opponent, surface=surface)
    if my_matches2.empty:
        print("No match found between " + player2 + " and " + opponent)
    else:
        print("%d matches between %s and %s" % (len(my_matches2), player2, opponent))
        
    df_points_selected2 = select_players_points(point_data=point_data, match_data=my_matches2)
    
    if (df_points_selected1.empty == False) & (df_points_selected2.empty == False):      
        players = [player1, player2]
        serve_sides = ["deuce", "ad"]
        serve_seqs = ["Sv1_direction", "Sv2_direction"]
        surfaces = ["hard", "clay", "grass"]
        
        print(surface)
        for serve_side in serve_sides:
            print(serve_side)
            for serve_seq in serve_seqs:
                print(serve_seq)
                contingency_table = pd.DataFrame()
                
                serve_dir_counts1 = df_points_selected1.loc[(df_points_selected1["server_name"].str.contains(player1)) & 
                                                          (df_points_selected1["serve_side"] == serve_side), serve_seq].value_counts()
                serve_dir_counts1["server_name"] = player1
                contingency_table = contingency_table.append(serve_dir_counts1, ignore_index=True)

                serve_dir_counts2 = df_points_selected2.loc[(df_points_selected2["server_name"].str.contains(player2)) & 
                                                          (df_points_selected2["serve_side"] == serve_side), serve_seq].value_counts()
                serve_dir_counts2["server_name"] = player2
                
                contingency_table = contingency_table.append(serve_dir_counts2, ignore_index=True)
            
                contingency_table.set_index("server_name", inplace=True)
                contingency_table.fillna(0, inplace=True)
                
                contingency_table = contingency_table[["wide", "body", "t"]]
                print(contingency_table)
                
                if test_mode == "freeman-halton":
                    freeman_halton_test(contingency_table.to_numpy())
                elif test_mode == "chi-square":
                    chi_square(contingency_table)
                print()
    else:
        print("No match found")

In [ ]:
player1 = "federer"
player2 = "nadal"
opponent = "djokovic"
surface = "grass"

compare_player_serves_when_they_play_same_opponent(point_data = df_points, match_data = df_matches, player1 = player1, player2 = player2, 
                                                   opponent = opponent, surface = surface, test_mode="chi-square")

In [ ]:
# Find out how many matches two players played against each other
# Use the match for checking internal consistency

player1 = "djokovic"
player2 = "murray"
tournament = "australian"
date = "2019"

my_matches = select_matches(data=df_matches, player1=player1, player2=player2, tournament=tournament, date=None)

print(my_matches)

## Check the internal consistency of a player's serve patterns

In [263]:
# Check whether a player's serve pattern is consistent within a match. 
# Retrieve all the points each player served, randomly split them into half. 
# Count the frequency of serve directions for each half and conduct a Chi-square test to see they are significantly different, controlled by 
# serve sides and serve sequences.
# We can do the random split multiple times to check the results. 

# To-do: use Freeman-Halton test (2x3 table, Fisher's exact test extension) to compare the two samples because the sample
# size is often very small and there are 0s in the data. 
# A chi-square test is performed on the data set, providing that at least 80% of the cells have an expected frequency of 5 or greater, and that no cell has an expected frequency smaller than 1.0. 
# Do not use Chi-square test. 
# Online Freeman-Halton tests can be found at http://vassarstats.net/fisher2x3.html and 
# https://www.danielsoper.com/statcalc/calculator.aspx?id=58. Use them for verification.
# 
def check_internal_consistency_split_half(point_data, match_id, player1):
    df_points_selected = point_data.loc[point_data["match_id"] == match_id]
    
    return_value = pd.DataFrame()
    
    min_point_count = 20
    # Some matches, such as "20150208-M-Montpellier-F-Richard_Gasquet-Jerzy_Janowicz"
    # have only a small number of points. Ignore them. 
    if len(df_points_selected) < min_point_count:
        print("Error: %s has fewer than %d points" % (match_id, min_point_count))
        print()
        return return_value
    
    for player in [player1]:
#         print(player)
        
        # Get all the points served by the current player
        df_points_player_served = df_points_selected.loc[df_points_selected["server_name"].str.contains(player), 
                                                          ["serve_side", "server_name", "Sv1_direction", "Sv2_direction"]]

        # Split the points in half, randomly
        df_points_player_served_half1 = df_points_player_served.sample(frac=0.5)
        df_points_player_served_half2 = df_points_player_served.drop(df_points_player_served_half1.index)
    #     print(df_points_player1_served_half1)
    #     print(df_points_player1_served_half2)

        # Divide each half by serve side and serve sequence
        for serve_side in ["deuce", "ad"]:
#             print(serve_side)
            for serve_seq in ["Sv1_direction", "Sv2_direction"]:

                contingency_table = pd.DataFrame(columns=["group", "wide", "body", "t"])

#                 print(serve_seq)
                
                # Count the frequency of serve directions for the first half
                serve_dir_counts1 = df_points_player_served_half1.loc[df_points_player_served_half1["serve_side"] \
                                                                       == serve_side, serve_seq].value_counts(dropna=False)
                
                # Remove unknown serve directions
                serve_dir_counts1.drop(labels = "unknown", inplace=True, errors="ignore")
                serve_dir_counts1["group"] = 1
                
                # Add to the contingency table
                contingency_table = contingency_table.append(serve_dir_counts1)
    #             print(serve_dir_counts1)

                # Count the frequency of serve directions for the second half
                serve_dir_counts2 = df_points_player_served_half2.loc[df_points_player_served_half2["serve_side"] \
                                                                       == serve_side, serve_seq].value_counts(dropna=False)
                serve_dir_counts2.drop(labels = "unknown", inplace=True, errors="ignore")
                serve_dir_counts2["group"] = 2
                contingency_table = contingency_table.append(serve_dir_counts2)
    #             print(serve_dir_counts2)

                # Use Chi-square test to compare the distribution of the two halves
                contingency_table.set_index("group", inplace=True)
                # If a column is all na, drop it. Otherwise, Chi-square test will report error. 
#                 contingency_table.dropna(axis=1, how="all", inplace=True)
                contingency_table.fillna(0, inplace=True)
#                 print(contingency_table)
                
                p_value = freeman_halton_test(contingency_table.to_numpy())
                entry = pd.Series({"player": player, "serve_side": serve_side, "serve_sequence": serve_seq, "p-value": p_value})
                return_value = return_value.append(entry, ignore_index=True)
                
    return return_value
                
                # The sample sizes are often too small for chi-square test
                # chi_square(contingency_table)
#                 print()

In [265]:
# Check for internal consistency for one player over many matches

for player in player_match_count[player_match_count >= 30].index:
    print(player)

    player_matches = select_matches(data=df_matches, player1=player)
    player_match_consistency = pd.DataFrame()
    for match_id in player_matches["match_id"]:
#         print(match_id)
        player_consistency_table = check_internal_consistency_split_half(df_points, match_id, player)
        if player_consistency_table.empty != True:
            player_consistency_table["match_id"] = match_id
            player_match_consistency = player_match_consistency.append(player_consistency_table, ignore_index=True)

player_match_consistency.to_excel("player_match_serve_consistency.xls")

roger_federer
novak_djokovic
rafael_nadal
andy_murray
stefan_edberg
pete_sampras
andre_agassi
lleyton_hewitt
juan_martin_del_potro
stanislas_wawrinka
ivan_lendl
boris_becker
gael_monfils
david_ferrer
alexander_zverev
dominic_thiem
andy_roddick
tomas_berdych
milos_raonic
karim_mohamed_maamoun
kei_nishikori
grigor_dimitrov
jo_wilfried_tsonga
daniil_medvedev
richard_gasquet
Error: 20150208-M-Montpellier-F-Richard_Gasquet-Jerzy_Janowicz has fewer than 20 points

john_isner
stefanos_tsitsipas
nick_kyrgios
robin_haase
philipp_kohlschreiber
denis_shapovalov
borna_coric
roberto_bautista_agut


In [267]:
pct_H0 = 100 * len(player_match_consistency.loc[player_match_consistency["p-value"] > 0.05])/len(player_match_consistency)

print("%f%% of matches have internal serve pattern consistency" % pct_H0)

99.166667% of matches have internal serve pattern consistency


## Check the external consistency of a player's serve pattern using parallel forms reliability test

In [121]:
# Find how many matches a player played in a tournament. 
# The more matches in a tournament the better for checking external consistency using parallel forms reliability test

# Need at least four matches per tournament to test external consistency
# Djokovic 2016 AO 2019(3), 2016(4), 2012(4), wimbledon 2019(6), 2018(3), 2015 (3), french open 2015(3)
# Federer 2020 AO (4), Wimbledon 2019(6), 2018(5), 2017(3)  
# Nadal AO 2014(6), 2017(3), 2019(2), Wimbledon 2019(5), 2018(2), 2008(3), Roland Garros 2014(7)
# Wawrinka US Open 2016(3), AO 2014 (3)
# Murray AO 2012 (3), 2020 (3), US Open 2012 (3), 
# Sampras: US Open 2001(3), AO 1995(3)
# del Potro: French 2019 (3), 2018(3), us open 2018(3), 2009(3)
# Agassi: us open 1995(3), 2005(3)

player = "federer"
tournament = "wimbledon"
date = None

my_matches = select_matches(data=df_matches, player1=player, tournament=tournament, date=date)

print(my_matches)

                                               match_id            Player 1  \
244   20190714-M-Wimbledon-F-Roger_Federer-Novak_Djo...       roger_federer   
245   20190712-M-Wimbledon-SF-Roger_Federer-Rafael_N...       roger_federer   
247   20190710-M-Wimbledon-QF-Roger_Federer-Kei_Nish...       roger_federer   
250   20190708-M-Wimbledon-R16-Roger_Federer-Matteo_...       roger_federer   
256   20190706-M-Wimbledon-R32-Roger_Federer-Lucas_P...       roger_federer   
263   20190704-M-Wimbledon-R64-Roger_Federer-Jay_Clarke       roger_federer   
663   20180711-M-Wimbledon-QF-Kevin_Anderson-Roger_F...      kevin_anderson   
664   20180709-M-Wimbledon-R16-Adrian_Mannarino-Roge...    adrian_mannarino   
665   20180706-M-Wimbledon-R32-Roger_Federer-Jan_Len...       roger_federer   
666   20180704-M-Wimbledon-R64-Lukas_Lacko-Roger_Fed...         lukas_lacko   
667   20180702-M-Wimbledon-R128-Dusan_Lajovic-Roger_...       dusan_lajovic   
790    20170716-M-Wimbledon-F-Marin_Cilic-Roger_Fede

In [47]:
def check_external_consistency_parallel_forms_reliability(point_data, match_data1, match_data2, player):
    selected_point_data1 = select_players_points(point_data=point_data, match_data=match_data1)
    selected_point_data1 = selected_point_data1.loc[selected_point_data1["server_name"].str.contains(player)]
    
    selected_point_data2 = select_players_points(point_data=point_data, match_data=match_data2)
    selected_point_data2 = selected_point_data2.loc[selected_point_data2["server_name"].str.contains(player)]
    
    print(player)
    
    for serve_side in ["deuce", "ad"]:
        print(serve_side)
        for serve_seq in ["Sv1_direction", "Sv2_direction"]:
            print(serve_seq)
            
            contingency_table = pd.DataFrame(columns=["match", "wide", "body", "t"])
            
            serve_dir_counts1 = selected_point_data1.loc[selected_point_data1["serve_side"] == serve_side, serve_seq].value_counts(dropna=False)
            serve_dir_counts1["match"] = 1
            serve_dir_counts1.drop(labels = "unknown", inplace=True, errors="ignore")
            
            contingency_table = contingency_table.append(serve_dir_counts1)
            
            serve_dir_counts2 = selected_point_data2.loc[selected_point_data2["serve_side"] == serve_side, serve_seq].value_counts(dropna=False)
            serve_dir_counts2["match"] = 2
            serve_dir_counts2.drop(labels = "unknown", inplace=True, errors="ignore")
            
            contingency_table = contingency_table.append(serve_dir_counts2)
            
            contingency_table.set_index("match", inplace=True)
            # If a column is all na, drop it. Otherwise, Chi-square test will report error.
#             contingency_table.dropna(axis=1, how="all", inplace=True)
            contingency_table.fillna(0, inplace=True)
            print(contingency_table)
            freeman_halton_test(contingency_table.to_numpy())
            
            # The sample sizes are often too small for chi-square test
            # chi_square(contingency_table)
            print()
    

In [52]:
# Year by year comparison found more inconsistent server patterns but still many are consistent. 
# Within tournament consistency is generally very high.

player = "federer"
opponent1 = "djokovic"
opponent2 = "millman"
tournament = "australian"
date1 = "2020"
date2 = "2020"

my_matches1 = select_matches(data=df_matches, player1=player, player2 = opponent1, tournament=tournament, date=date1)
print(str(len(my_matches1)) + " matches")

my_matches2 = select_matches(data=df_matches, player1=player, player2 = opponent2, tournament=tournament, date=date2)
print(str(len(my_matches2)) + " matches")

# print(match_id1 + " " + match_id2)

check_external_consistency_parallel_forms_reliability(point_data = df_points, 
                                                      match_data1 = my_matches1, 
                                                      match_data2 = my_matches2, 
                                                      player = player)


1 matches
1 matches
federer
deuce
Sv1_direction
       wide  body     t
match                  
1.0    30.0   0.0  22.0
2.0    44.0   2.0  40.0
Freeman-Halton Test
p = 0.549067376698846
Independent (fail to reject H0)

Sv2_direction
       wide  body  t
match               
1         6     4  5
2        12    10  6
Freeman-Halton Test
p = 0.7196135395075792
Independent (fail to reject H0)

ad
Sv1_direction
       wide  body     t
match                  
1.0    26.0   0.0  26.0
2.0    42.0   3.0  30.0
Freeman-Halton Test
p = 0.2440413219043995
Independent (fail to reject H0)

Sv2_direction
       wide  body  t
match               
1         7     8  6
2        12     9  7
Freeman-Halton Test
p = 0.8156386818235442
Independent (fail to reject H0)



To-do

1. Serve direction analysis
The characteristics of serves for one player
    - Comparison by serve sides(use chi-square)
    - Comparison first and second serves(use chi-square, ind=serves, dep=serve_direction)
    - Comparison by surfaces (control by opponent)(use chi-square)
    - Comparison by opponents (different serve patterns for different opponents?) (use chi-square)
    - Comparison by age (early years vs later years) (chi-square, ind=different ages, dep=serve_directions)
    - Comparison by tournament (grand slam vs other tournaments) (chi-square)
    - Show the change of serve directions during a match (draw a line chart show the counts of each serve direction over time)
    - Serve patterns at critical moments vs non-critical moments (1 points from winning, 1 points from losing, etc.)
    - Serve pattern based on tiredness, group serves by the count of points played (early, middle, or later) and compare between groups.
    - Serve patterns when leading vs when trailing
    - Serve patterns when first serve rate is high (high confidence) vs when the rate is low (low confidence)
    - Serve patterns when winning several points in a row (high confidence), serve patterns when losing several points in a row (low confidence, frustration). 
    - Serve after unforced errors (one or more). Serve after the opponent hitting a winner. Serve after hitting winners. 
    - Serve right after long points. 
    - Serve right after aces (is there a difference? Is he likely to serve a different direction?)
    - Serve right after double faults (is there a difference?)
    - Serve right after repeated first serve faults
    - Is the success or failure of certain serve directions (e.g., aces, short points won, double faults) in the beginning influence the later serve decisions? (anchor effect) Any short term or long term effects?
    - Can we find any subtle bias in serve selection?
    
Compare the serves of two players
    - When they play each other ...
    - When they play the same opponent ...
    - When they are tired
    - When they are leading or losing
    - At critical moments ...
    - At high or low confidence level 

Compare the serves for three or more players?
    - Is it useful?
    
2. Serve error analysis
    - Frequency of errors at critical moments (double fault, first serve fault)
    - Types of errors correlated with tension, confidence, etc. 


In [ ]:
t